In [12]:
import cotengra as ctg
from _formula import *
from _sat import *

## Boolean operations as tensors

Rank-1 tensor for output nodes.

In [13]:
output_t: np.ndarray = np.array([0, 1])

Rank-2 tensor for negation.

In [14]:
negation_t: np.ndarray = np.array([[0, 1], [1, 0]])

Rank-3 tensor for disjunction.

In [15]:
disj_t: np.ndarray = np.zeros((2, 2, 2))

for i in range(2):
    for j in range(2):
        disj_t[i][j][i | j] = 1

## Input arrays

The `input_arrays` function constructs the tensors that the tensor network representation of the CNF will be instantiated with to perform the contraction.

In [16]:
def input_arrays(cnf: CNFInstance) -> List[np.ndarray]:
    """Constructs the tensors on which the contraction will be instantiated
       to find a satisfying assignment."""
    inputs = cnf.network().wiring.hypergraph.inputs
    arrays = []

    for n in inputs:
        if len(n) == 1:
            arrays.append(output_t)
        elif len(n) == 2:
            arrays.append(negation_t)
        elif len(n) == 3:
            arrays.append(disj_t)

    return arrays

## Translating CNF formulae to Einsum expressions

In [17]:
def einsum_expression(cnf: CNFInstance):
    """Einsum expression corresponding to a given CNF formula."""
    graph = cnf.network().wiring.hypergraph
    expr = ctg.utils.inputs_output_to_eq(graph.inputs, graph.output)

    [inputs, outputs] = expr.split("->")

    n = max([ int(n) for n in inputs if n != "," ])

    for i in range(n+1):
        inputs = inputs.replace(str(i), str(chr(97 + i)))
        outputs = outputs.replace(str(i), str(chr(97 + i)))

    return inputs + "->" + outputs

## Testing out an example

In [18]:
cnf1: CNFInstance = CNFInstance.random(k=3, n=3, m=2, rng=0)
cnf2: CNFInstance = CNFInstance.from_formula(neg(conj(disj(variable("a"), variable("b")), conj(neg(variable("a")), neg(variable("b"))))))

print(cnf1.clauses)

expr1: str = einsum_expression(cnf1)
expr2: str = einsum_expression(cnf2)

print(expr1)

((np.int64(1), np.int64(2), np.int64(3)), (np.int64(-3), np.int64(-2), np.int64(-1)))
abd,dce,e,cf,bg,ah,fgi,ihj,j->abc


`abd,dce,e,cf,bg,ah,fgi,ihj,j->abc`

- `d = a | b`
- `e = d | c`
- `assert e == 1`
- `f = ~ c`
- `g = ~ b`
- `h = ~ a`
- `i = f | g`
- `j = f | h`
- `assert j == 1`
- `return a , b , c`


In [22]:
[ int(x) for x in ctg.einsum(expr1, *input_arrays(cnf1)).reshape(8) ]

[0, 1, 1, 1, 1, 1, 1, 0]